In [233]:
#import basic Libraries
import pandas as pd
import numpy as np
from tqdm import tqdm

#Visualizaiton imports
import seaborn as sns
import matplotlib.pyplot as plt

#Importing Sklearn
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#Import Market Basket Models
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

%matplotlib inline

In [234]:
#import of all files from the data folder.
aisles = pd.read_csv('./data/aisles.csv')
departments = pd.read_csv('./data/departments.csv')
products = pd.read_csv('./data/products.csv')
order_products__prior = pd.read_csv('./data/order_products__prior.csv')
order_products__train = pd.read_csv('./data/order_products__train.csv')
orders = pd.read_csv('./data/orders.csv')
merged_products_aisles_departments = pd.read_csv('./data/merged_data.csv')
merged_products_aisles_departments = merged_products_aisles_departments.drop('Unnamed: 0', axis=1)

# Preprocessing

In [294]:
opt = order_products__train.merge(merged_products_aisles_departments, how='left', on='product_id')

In [295]:
opt.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,foodgroup
0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs,diary products and alternatives
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs,diary products and alternatives
2,1,10246,3,0,Organic Celery Hearts,83,4,fresh vegetables,produce,vegetables and fruits
3,1,49683,4,0,Cucumber Kirby,83,4,fresh vegetables,produce,vegetables and fruits
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods,others


In [296]:
#converting the id to string to preserve order.
opt['aisle_id']= opt['aisle_id'].astype(str)

In [297]:
#instanciate Oone hot encoder ot break out the data my aisle.
ohe = OneHotEncoder(handle_unknown='ignore')

In [298]:
#test to  see # of aisles
print('# of unique aisles =',len(opt['aisle_id'].unique()))
aisle = ohe.fit_transform(opt['aisle_id'].astype(str).values.reshape(-1,1))

# of unique aisles = 134


In [299]:
#one hot encode the aisles and create a dense matrix.
a_ohe = pd.DataFrame(aisle.toarray())
aisle = a_ohe.to_dense()

In [300]:
aisle

,0,1,2,3,4,5,6,7,8,9,...,124,125,126,127,128,129,130,131,132,133
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [301]:
#add order_id to the one hot encoded matrix
basket = opt[['order_id']].join(aisle)

In [304]:
basket.head()

,order_id,0,1,2,3,4,5,6,7,8,...,124,125,126,127,128,129,130,131,132,133
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [305]:
range(len(opt['aisle_id'].unique()))

range(0, 134)

In [306]:
#return the total items in the basket from the aisles
basket = basket.groupby('order_id').sum()

In [308]:
basket.head()

,0,1,2,3,4,5,6,7,8,9,...,124,125,126,127,128,129,130,131,132,133
order_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [309]:
#covert any values to be a binary. 
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [311]:
basket_sets.head()

,0,1,2,3,4,5,6,7,8,9,...,124,125,126,127,128,129,130,131,132,133
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0


In [312]:
aisle_dict = dict(zip(opt['aisle_id'].unique().astype('Int64'),opt['aisle'].unique()))

/Users/soeunjang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  """Entry point for launching an IPython kernel.


In [313]:
basket_sets.columns = list(range(1,len(aisle_dict)+1))
basket_sets.rename(columns=aisle_dict,inplace=True)

In [314]:
basket_sets.head()

,prepared soups salads,specialty cheeses,energy granola bars,instant foods,marinades meat preparation,other,packaged meat,bakery desserts,pasta sauce,kitchen supplies,...,trail mix snack mix,feminine care,body lotions soap,tortillas flat bread,frozen appetizers sides,hot cereal pancake mixes,dry pasta,beauty,muscles joints pain relief,specialty wines champagnes
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0


In [315]:
#creates conditions for the items in the basket.
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

Apriori is an algorithm for extracting frequent itemsets with applications in association rule learning. The apriori algorithm has been designed to operate on databases containing transactions, such as purchases by customers of a store. An itemset is considered as "frequent" if it meets a user-specified support threshold. For instance, if the support threshold is set to 0.5, a frequent itemset is defined as a set of items that occur together in at least 50% of all transactions in the database.

# Market Basket Results

In [250]:
#creates the rules to filter.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [251]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(fresh fruits),(fresh herbs),0.550099,0.096868,0.074446,0.135332,1.397072,0.021159,1.044484
1,(fresh herbs),(fresh fruits),0.096868,0.550099,0.074446,0.768529,1.397072,0.021159,1.943656
2,(fresh vegetables),(fresh herbs),0.450975,0.096868,0.083470,0.185088,1.910712,0.039785,1.108256
3,(fresh herbs),(fresh vegetables),0.096868,0.450975,0.083470,0.861684,1.910712,0.039785,3.969343
4,(packaged cheese),(fresh fruits),0.237781,0.550099,0.160911,0.676720,1.230178,0.030108,1.391676


Support is the relatvie frequency that the rules show up.
Confidence is a measure of the reliability of the rule.
Lift is the ratio of the observed support to that expected if the two rules were independet.

In [252]:
#sorts to see the lift greater than or equal to 1
rules[rules['lift'] >= 1.5].head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(fresh vegetables),(fresh herbs),0.450975,0.096868,0.083470,0.185088,1.910712,0.039785,1.108256
3,(fresh herbs),(fresh vegetables),0.096868,0.450975,0.083470,0.861684,1.910712,0.039785,3.969343
60,(yogurt),(milk),0.253405,0.223750,0.086991,0.343289,1.534251,0.030292,1.182026
61,(milk),(yogurt),0.223750,0.253405,0.086991,0.388787,1.534251,0.030292,1.221497
79,"(packaged cheese, fresh fruits)",(fresh vegetables),0.160911,0.450975,0.112538,0.699380,1.550816,0.039971,1.826306


# Market Basket for Products

In [318]:
reorders = opt[opt['reordered'] == 1]
#opt[opt['reordered'] == 1]

In [319]:
reorders['product_id'] = reorders['product_id'].astype('int64')

/Users/soeunjang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [320]:
reorders.dtypes

order_id              int64
product_id            int64
add_to_cart_order     int64
reordered             int64
product_name         object
aisle_id             object
department_id         int64
aisle                object
department           object
foodgroup            object
dtype: object

In [342]:
# get list of hi volume products
hivol = reorders['product_id'].value_counts().sort_values(ascending=False)[reorders['product_id'].value_counts().sort_values(ascending=False) > 1]

In [347]:
# mask the reorders dataframe to only incluse dem hi
reorders = reorders[reorders['product_id'].isin(hivol)]
reorders

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,foodgroup
51,98,329,20,1,Organic Whole Grassmilk Milk,84,16,milk,dairy eggs,diary products and alternatives
227,915,196,3,1,Soda,77,7,soft drinks,beverages,others
547,1890,432,1,1,Vanilla Almond Breeze Almond Milk,91,16,soy lactosefree,dairy eggs,diary products and alternatives
654,2021,199,11,1,Herb Thyme Clamshell,16,4,fresh herbs,produce,vegetables and fruits
809,2822,260,9,1,Cantaloupe,24,4,fresh fruits,produce,vegetables and fruits
975,3160,196,1,1,Soda,77,7,soft drinks,beverages,others
1620,4305,195,1,1,Grade A Pasteurized 2% Milkfat Lowfat Cottage ...,108,16,other creams cheeses,dairy eggs,diary products and alternatives
1726,4468,260,4,1,Cantaloupe,24,4,fresh fruits,produce,vegetables and fruits
1767,4562,117,4,1,Petit Suisse Fruit,2,16,specialty cheeses,dairy eggs,diary products and alternatives
1768,4562,45,5,1,European Cucumber,83,4,fresh vegetables,produce,vegetables and fruits


In [258]:
reorders['hi_dem'] = (reorders.copy()['product_id'].value_counts().sort_values(ascending=False)>1)

In [259]:
hidem_ord = reorders[reorders['hi_dem'] == True]

In [289]:
hidem_ord['product_id'] = hidem_ord.copy()['product_id'].astype(str)

/Users/soeunjang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [290]:
#tranposes the data to ensure that it is correctly fitted.
product = ohe.fit_transform(hidem_ord['product_id'].values.reshape(-1,1))

In [292]:
p_ohe = pd.DataFrame(product.toarray())
products = p_ohe.to_dense()

In [274]:
basket = hidem_ord[['order_id']].join(products)
basket

,order_id,0,1,2,3,4,5,6,7,8,...,4813,4814,4815,4816,4817,4818,4819,4820,4821,4822
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29,96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32,98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35,98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [283]:
#return the total items in the basket from the aisles
basket = basket.groupby('order_id').sum()
basket

,0,1,2,3,4,5,6,7,8,9,...,4813,4814,4815,4816,4817,4818,4819,4820,4821,4822
order_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [265]:
#covert any values to be a binary.
def encode_units(x):
    if x <= 0 or 'NaN':
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [266]:
pro_dict = dict(zip(hidem_ord['product_id'].unique().astype('Int64'),hidem_ord['product_name'].unique()))

/Users/soeunjang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  """Entry point for launching an IPython kernel.


In [267]:
pro_dict = dict(zip(list(range(0,len(hidem_ord['product_id'].unique()))),hidem_ord['product_name'].unique()))

In [268]:
#rename the columns for easier analysis in rules. 
basket_sets.columns = list(range(0,len(pro_dict)))
basket_sets.rename(columns=pro_dict,inplace=True)

In [269]:
basket_sets.sum().head()

Organic 4% Milk Fat Whole Milk Cottage Cheese    1.0
Lightly Smoked Sardines in Olive Oil             0.0
Spring Water                                     2.0
Cage Free Extra Large Grade AA Eggs              0.0
Roasted Turkey                                   0.0
dtype: float64

In [270]:
#creates conditions for the items in the basket.
frequent_itemsets = apriori(basket_sets, min_support=0.0005, use_colnames=True)

ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value nan